In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

C:\Users\Thier\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y%m%d")

In [5]:
df['y/m'] = pd.to_datetime(df['y/m']).dt.strftime("%Y%m%d")

In [6]:
X = df.drop(['3_sigma_event'], axis=1).values
Y = df['3_sigma_event'].values

print(X.shape, Y.shape)

(7345, 49) (7345,)


In [7]:
nrows=7345
n_train = int(nrows * 0.9)
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

In [8]:
n_features = int(X_train.shape[1])
learning_rate = 0.001
n_iter = 50

In [9]:
# Input and Target placeholders
x = tf.placeholder(tf.float32, shape=[None, n_features])
y = tf.placeholder(tf.float32, shape=[None])

In [10]:
# Build the logistic regression model
W = tf.Variable(tf.zeros([n_features, 1]))
b = tf.Variable(tf.zeros([1]))

In [11]:
logits = tf.add(tf.matmul(x, W), b)[:, 0]
pred = tf.nn.sigmoid(logits)

In [12]:
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits))
auc = tf.metrics.auc(tf.cast(y, tf.int64), pred)[1]

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
# Initialize the variables
init_vars = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [15]:
batch_size = 1000

In [16]:
indices = list(range(n_train))

In [17]:
def gen_batch(indices):
    np.random.shuffle(indices)
    for batch_i in range(int(n_train / batch_size)):
        batch_index = indices[batch_i*batch_size: (batch_i+1)*batch_size]
        yield X_train[batch_index], Y_train[batch_index]

In [18]:
sess = tf.Session()

sess.run(init_vars)

In [19]:
for i in range(1, n_iter+1):
    avg_cost = 0.
    for X_batch, Y_batch in gen_batch(indices):
        _, c = sess.run([optimizer, cost], feed_dict={x: X_batch, y: Y_batch})
        avg_cost += c / int(n_train / batch_size)
    print('Iteration %i, training loss: %f' % (i, avg_cost))

Iteration 1, training loss: 892549.246709
Iteration 2, training loss: 1757647.458659
Iteration 3, training loss: 2184871.138021
Iteration 4, training loss: 2271129.110352
Iteration 5, training loss: 2435634.638346
Iteration 6, training loss: 2462992.256836
Iteration 7, training loss: 1886568.710938
Iteration 8, training loss: 1893503.339518
Iteration 9, training loss: 2383397.956055
Iteration 10, training loss: 2247837.106608
Iteration 11, training loss: 2168316.942708
Iteration 12, training loss: 2167923.968913
Iteration 13, training loss: 1832996.913411
Iteration 14, training loss: 2004902.873047
Iteration 15, training loss: 1686979.290690
Iteration 16, training loss: 1395866.365560
Iteration 17, training loss: 1743437.205078
Iteration 18, training loss: 816248.310059
Iteration 19, training loss: 1574063.158854
Iteration 20, training loss: 1133622.151855
Iteration 21, training loss: 1356809.097412
Iteration 22, training loss: 946734.137044
Iteration 23, training loss: 1181189.739258


In [20]:
auc_test = sess.run(auc, feed_dict={x: X_test, y: Y_test})
print("AUC of ROC on testing set:", auc_test)

AUC of ROC on testing set: 0.5000001
